In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import timeDistance
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")

In [2]:
new_route(fur1["mercoledi"].dropna(), dft)

Castelverde, Ubaldo Ferrari, 15 $ Annicco, Giuseppe Garibaldi, 6         15.0
Annicco, Giuseppe Garibaldi, 6 $ Castelleone, Roma, 67                   14.0
Castelleone, Roma, 67 $ Madignano, Portici, 4                             9.0
Madignano, Portici, 4 $ Crema, Civerchi, 9                                8.0
Crema, Civerchi, 9 $ Palazzo Pignano, Marco Polo, 31                     13.0
Palazzo Pignano, Marco Polo, 31 $ Pandino, Castello, 15                   7.0
Pandino, Castello, 15 $ Rivolta d'Adda, Stefano Renzi, 5                  9.0
Rivolta d'Adda, Stefano Renzi, 5 $ Spino d'Adda, Circonvallazione, 13    10.0
Spino d'Adda, Circonvallazione, 13 $ Romanengo, Guaiarini, 14            25.0
dtype: float64

In [3]:
def definitive_local_search(fur, n, dft):
    #Schedulazione iniziale, e sequenza dei nomi iniziali, una per ciclare e una copia per effettuare pop e insert
    pre_schedule = new_route(fur, dft)
    lista = [getName(dft, x) for x in fur]
    sequence = lista[:]
    #Costo iniziale del percorso greedy in base al furgoncino "n"
    if(n==1):
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'Ostiano, Mazzini, 1', dft)
    else:
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
    new_cost = 0
    local_best_cost = math.inf
    local_best_schedule = []

    #Eseguo finchè ho dei miglioramenti e quindi i nuovi costi sono migliori del migliore al passo prima
    while(True):
        #Prendo ogni città in lista, la inserisco nella posizione migliore, e tra tutte le città prendo quella che da lo
        #spostamento più vantaggioso.
        #Se questo è anche migliore del risultato precedente utilizzo la nuova sequenza come passo del ciclo successivo
        for i in range(len(lista)):
            #Copia della lista su cui lavorare
            sequence = lista[:]
            to_insert = lista[i]
            sequence.remove(to_insert)
            curr_schedule = new_route(sequence, dft)
            index = best_index(sequence, curr_schedule, to_insert, dft)
            try:
                sequence.insert(index, to_insert)
            except:
                print(sequence)
                print(to_insert)
                raise Exception("Error on insert")
            curr_schedule = new_route(sequence, dft)
            if(n==1):
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'Ostiano, Mazzini, 1', dft)
            else:
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
            if(new_cost<local_best_cost):
                local_best_cost = new_cost
                local_best_schedule = curr_schedule[:]
        if(local_best_cost<pre_cost):
            pre_cost = local_best_cost
            pre_schedule = local_best_schedule[:]
            lista = cities_list(pre_schedule)
        else:
            break
    return cities_list(pre_schedule), pre_cost

In [4]:
#TIME LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna(), 1, dft)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    print(sch)
    to_write[day] = pd.Series(sch)

['Cremona, Sesto 39', 'Sesto ed Uniti, Roma, 4', 'Grumello Cremonese ed Uniti, Roma, 2', 'Pizzighettone, Giuseppe Garibaldi, 18', 'Soresina, Giacomo Matteotti, 6', 'Trigolo, Roma, 26', 'Castelleone, Roma, 67', 'Montodine,  XXV Aprile, 10', 'Ripalta Cremasca, Roma, 5', 'Ricengo, Guglielmo Marconi', 'Capralba, Piave, 2', 'Sergnano, IV Novembre,  10', 'Casale Cremasco-Vidolasco, Roma, 56', 'Camisano, Trieste, 57', 'Soncino, Dante Alighieri, 23', 'Ostiano, Mazzini, 1']
['Cremona, Sesto 39', 'Cremona, Ugolani Dati, 4', 'Persico Dosimo, Albert ,7', 'Ostiano, Mazzini, 1', 'San Giovanni in Croce, Feudatari, 25 A', 'Casalmaggiore, Guglielmo Marconi, 8', 'Scandolara Ravara, Italia', 'Casalpusterlengo, Pusterla 7', 'Vailate, Dante Alighieri, 7', 'Crema, Civerchi, 9', 'Ostiano, Mazzini, 1']
['Cremona, Sesto 39', 'Castelverde, Ubaldo Ferrari, 15', 'Annicco, Giuseppe Garibaldi, 6', 'Castelleone, Roma, 67', 'Madignano, Portici, 4', 'Crema, Civerchi, 9', 'Palazzo Pignano, Marco Polo, 31', 'Pandino, Ca

In [5]:
to_write.to_csv("../Dati/turni_f/ls1_shifts.csv", index=False, sep=";")

In [6]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur2[day].dropna(), 2, dft)
    print(day)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_shifts.csv", index=False, sep=";")

martedi
mercoledi
giovedi


In [7]:
#COMPUTE KM LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1d[day].dropna(), 1, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls1_km_shifts.csv", index=False, sep=";")

In [8]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur2d[day].dropna(), 2, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_km_shifts.csv", index=False, sep=";")